In [1]:
import open3d as o3d
import numpy as np
import fpsample

input_file = "centered.ply"
pcd = o3d.io.read_point_cloud(input_file)
vertices = np.asarray(pcd.points)
print("vertices shape:", vertices.shape)

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.
vertices shape: (395700, 3)


In [2]:
sampled_indices = fpsample.bucket_fps_kdtree_sampling(vertices, 10000)
sampled_pcd = pcd.select_by_index(sampled_indices)
print(sampled_indices)

[ 35743 285477  56311 ... 334376 368722 135843]


In [3]:
distances = np.linalg.norm(vertices[sampled_indices], axis=1)
normalized_distances = (distances - np.min(distances)) / (np.max(distances) - np.min(distances))
colors = 1 - np.repeat(normalized_distances[:, np.newaxis], 3, axis=1)
sampled_pcd.colors = o3d.utility.Vector3dVector(colors)

sampled_pcd.estimate_normals(search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=0.05, max_nn=30))
sampled_pcd.orient_normals_consistent_tangent_plane(k=30)

In [4]:
distances = sampled_pcd.compute_nearest_neighbor_distance()
avg_dist = np.mean(distances)
radii = o3d.utility.DoubleVector([avg_dist, avg_dist*1.5, avg_dist*3])
#Try to find the most trinangulated mesh
mesh = o3d.geometry.TriangleMesh.create_from_point_cloud_ball_pivoting(sampled_pcd, radii)
mesh = mesh.filter_smooth_taubin(number_of_iterations=30)
mesh.compute_triangle_normals()
mesh.compute_vertex_normals()

TriangleMesh with 10000 points and 10279 triangles.

In [5]:
o3d.io.write_triangle_mesh("output_mesh.ply", mesh)

o3d.visualization.draw_geometries(
    [mesh],
    mesh_show_back_face=True,
    mesh_show_wireframe=False,
    window_name="Optimalizált 3D Modell"
)